In [1]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('database.sqlite')

# Get the list of all tables in the database
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)['name'].tolist()

# Dictionary to hold dataframes
dfs = {}

# Read each table into a dataframe and store it in the dictionary
for table in tables:
    query = f"SELECT * FROM {table};"
    dfs[table] = pd.read_sql(query, conn)

# Close the connection
conn.close()

In [2]:
%load_ext sql
%sql sqlite:///database.sqlite

Let's list all the tables from the database.

In [3]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///database.sqlite
Done.


name
sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


Task 1: How many leagues in the dataset?

In [6]:
%%sql
SELECT COUNT(name) FROM League;

 * sqlite:///database.sqlite
Done.


COUNT(name)
11


Task 2: How many matches played in the league?

In [7]:
%%sql
SELECT
    l.name AS league,
    COUNT(m.country_id) as total_matches
FROM League AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
GROUP BY l.name;

 * sqlite:///database.sqlite
Done.


league,total_matches
Belgium Jupiler League,1728
England Premier League,3040
France Ligue 1,3040
Germany 1. Bundesliga,2448
Italy Serie A,3017
Netherlands Eredivisie,2448
Poland Ekstraklasa,1920
Portugal Liga ZON Sagres,2052
Scotland Premier League,1824
Spain LIGA BBVA,3040


Task 3: Find out the results in 2013-14 season.

In [20]:
%%sql
SELECT *
FROM Match
WHERE Season = '2013/2014'
LIMIT 1;

 * sqlite:///database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
1237,1,1,2013/2014,1,2014-03-29 00:00:00,1677179,8475,9989,2,0,1,2,4,6,8,2,4,6,8,4,6,1,3,5,7,2,4,6,8,3,4,7,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,7,7,7,7,10,10,10,208699,8930,208348,26606,186621,104410,32760,37963,376780,80592,466203,38953,45485,426892,237646,512033,36863,25655,466260,513810,389202,410160,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [16]:
%%sql
SELECT date, id, home_team_goal, away_team_goal
FROM Match
WHERE Season = '2013/2014'
LIMIT 3;

 * sqlite:///database.sqlite
Done.


date,id,home_team_goal,away_team_goal
2014-03-29 00:00:00,1237,2,0
2014-03-29 00:00:00,1238,0,1
2014-04-05 00:00:00,1239,1,0


Task 4: How many times home team won in that season?

In [25]:
%%sql
SELECT COUNT(*) AS home_team_wins
FROM Match
WHERE Season = '2013/2014'
AND home_team_goal > away_team_goal;

 * sqlite:///database.sqlite
Done.


home_team_wins
1404


Task 5: Make a column which shows the outcome of the match.

In [31]:
%%sql
SELECT date, home_team_goal, away_team_goal,
CASE WHEN home_team_goal > away_team_goal THEN 'HOME TEAM WINS'
     WHEN home_team_goal < away_team_goal THEN 'AWAY TEAM WINS' 
     ELSE 'TIE' END AS Outcome
FROM Match
WHERE Season = '2013/2014'
LIMIT 3;

 * sqlite:///database.sqlite
Done.


date,home_team_goal,away_team_goal,Outcome
2014-03-29 00:00:00,2,0,HOME TEAM WINS
2014-03-29 00:00:00,0,1,AWAY TEAM WINS
2014-04-05 00:00:00,1,0,HOME TEAM WINS


Task 6: In this exercise, you will identify matches played between FC Schalke 04 and FC Bayern Munich. There are 2 teams identified in each match in the hometeam_id and awayteam_id columns, available to you in the filtered matches_germany table. ID can join to the team_api_id column in the teams_germany table, but you cannot perform a join on both at the same time.

However, you can perform this operation using a CASE statement once you've identified the team_api_id associated with each team!



In [35]:
%%sql
SELECT *
FROM Team
WHERE team_long_name = 'FC Schalke 04' OR team_long_name = 'FC Bayern Munich'

 * sqlite:///database.sqlite
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
15617,9823,21,FC Bayern Munich,BMU
15621,10189,34,FC Schalke 04,S04


In [37]:
%%sql
SELECT team_api_id, team_long_name
FROM Team 
LEFT JOIN Match
ON Team.team_api_id = Match.id
WHERE team_long_name = 'FC Schalke 04' OR team_long_name = 'FC Bayern Munich'

 * sqlite:///database.sqlite
Done.


team_api_id,team_long_name
9823,FC Bayern Munich
10189,FC Schalke 04


In [43]:
%%sql
SELECT 
    CASE 
        WHEN Team.team_long_name = 'FC Bayern Munich' THEN 'FC Bayern Munich'
        WHEN Team.team_long_name = 'FC Schalke 04' THEN 'FC Schalke 04'
        ELSE 'Neither' 
    END AS home_team,
    COUNT(*) AS num_matches
FROM Match
JOIN Team ON Match.home_team_api_id = Team.team_api_id
WHERE Match.country_id = (SELECT id FROM Country WHERE name = 'Germany')
GROUP BY home_team;


 * sqlite:///database.sqlite
Done.


home_team,num_matches
FC Bayern Munich,136
FC Schalke 04,136
Neither,2176


# CASE statements comparing column values
Barcelona is considered one of the strongest teams in Spain's soccer league.

In this exercise, you will be creating a list of matches in the 2011/2012 season where Barcelona was the home team. You will do this using a CASE statement that compares the values of two columns to create a new group -- wins, losses, and ties.

In 3 steps, you will build a query that identifies a match's winner, identifies the identity of the opponent, and finally filters for Barcelona as the home team. Completing a query in this order will allow you to watch your results take shape with each new piece of information.

The matches_spain table currently contains Barcelona's matches from the 2011/2012 season, and has two key columns, hometeam_id and awayteam_id, that can be joined with the teams_spain table. However, you can only join teams_spain to one column at a time.

In [46]:
%%sql
/* step 1 */
SELECT 
/* Select the date of the match */
date,
    /* Identify home wins, losses, or ties */
CASE WHEN home_team_goal > away_team_goal THEN 'Home win!'
     WHEN home_team_goal < away_team_goal THEN 'Home loss :(' 
     ELSE 'Tie' END AS outcome
FROM Match
WHERE Match.country_id = (SELECT id FROM Country WHERE name = 'Germany')
LIMIT 4;

 * sqlite:///database.sqlite
Done.


date,outcome
2008-08-15 00:00:00,Tie
2008-08-16 00:00:00,Home loss :(
2008-08-16 00:00:00,Home win!
2008-08-16 00:00:00,Home win!


In [63]:
%%sql
/* Left join the teams_spain table team_api_id column to the matches_spain table awayteam_id. This allows us to retrieve the away team's identity.
Select team_long_name from teams_spain as opponent and complete the CASE statement from Step 1. */

SELECT 
    m.date,
    t.team_long_name AS opponent,
    CASE 
        WHEN m.home_team_goal > m.away_team_goal THEN 'Home win!'
        WHEN m.home_team_goal < m.away_team_goal THEN 'Home loss :('
        ELSE 'Tie' 
    END AS outcome
FROM Match AS m
LEFT JOIN Team AS t ON m.away_team_api_id = t.team_api_id
WHERE m.country_id = (SELECT id FROM Country WHERE name = 'Spain')
LIMIT 5;

 * sqlite:///database.sqlite
Done.


date,opponent,outcome
2008-08-30 00:00:00,RCD Mallorca,Home win!
2008-08-31 00:00:00,Villarreal CF,Tie
2008-08-31 00:00:00,Real Madrid CF,Home win!
2008-08-31 00:00:00,FC Barcelona,Home win!
2008-08-31 00:00:00,Sevilla FC,Tie


In [69]:
%%sql
/* Filter for matches where the home team is FC Barcelona (id = 8634). */
SELECT 
    m.date,
    t.team_long_name AS opponent,
    CASE 
        WHEN m.home_team_goal > m.away_team_goal THEN 'Barcelona win!'
        WHEN m.home_team_goal < m.away_team_goal THEN 'Barcelona loss :('
        ELSE 'Tie' 
    END AS outcome
FROM Match AS m
LEFT JOIN Team AS t ON m.away_team_api_id = t.team_api_id
WHERE m.country_id = (SELECT id FROM Country WHERE name = 'Spain')
AND m.home_team_api_id = 8634
LIMIT 4;

 * sqlite:///database.sqlite
Done.


date,opponent,outcome
2008-11-08 00:00:00,Real Valladolid,Barcelona win!
2008-11-23 00:00:00,Getafe CF,Tie
2008-12-06 00:00:00,Valencia CF,Barcelona win!
2008-12-13 00:00:00,Real Madrid CF,Barcelona win!
